- '03_웹크롤링_사이트 크롤링_네이버 뉴스1' 파일에서 수집한   
- 네이버 뉴스 섹션 메뉴와 섹션별 URL 활용해서  
(1) 섹션별 헤드라인 페이지 크롤링 : title, url 추출  
(2) (1)에서 추출한 url로 sub 페이지 내용 추출   
    - 각 기사 상세 내용 추출 : 언론사, 기사제목, 작성 날짜 시간, 기사 내용, 작성자

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
# !pip install pandas

In [3]:
from urllib.request import urlopen
import pandas as pd
# import bs4
from bs4 import BeautifulSoup
import requests

### (1) 네이버 뉴스 섹션별 헤드라인(topic) 페이지 크롤링

In [4]:
# 네이버 뉴스1에서 저장한 naver_news_section.csv 읽어오기
section_menu_df = pd.read_csv('./crawl_data/new_section_df.csv')

In [40]:
# 경제 섹션 링크 확인
section_menu_df[section_menu_df["menu"] == "경제"]

,menu,link
2,경제,https://news.naver.com/section/101


In [41]:
# url = section_menu_df[section_menu_df["menu"] == "경제"]["link"]
url = section_menu_df["link"][2]
url

'https://news.naver.com/section/101'

### 포털에서 제공하는 각 언론사 기사들의 카테고리인
- 정치~세계까지 6개 section에 대해 headline뉴스의 컨텐츠 수집
- 경제 섹션을 기준으로 코드 생성 후, 전체 수집 테스트
- 적용되지 않는 섹션의 코드만 수정해서 진행하는

In [42]:
html = urlopen(url)
bs_obj = BeautifulSoup(html, 'html.parser')

In [44]:
# 네이버 뉴스 섹션을 확인하면 
# 섹션별 헤드라인(topic)이 있고 
# topic의 title과 link를 크롤링할 예정

In [80]:
lis = bs_obj.select("ul.sa_list div.sa_text")
title = []
titleLink= []
for li in lis:
    title.append(li.a.text.replace("\n", ""))
    titleLink.append(li.a["href"])
len(title), len(titleLink)
naver_headline_link = pd.DataFrame({"title":title, "titleLink":titleLink})
naver_headline_link.head()

(46, 46)

,title,titleLink
0,포스코이앤씨 대표이사 사장에 정희민 건축사업본부장 선임,https://n.news.naver.com/mnews/article/366/000...
1,내년 실손보험료 평균 7.5% 인상,https://n.news.naver.com/mnews/article/082/000...
2,"소비심리, 계엄 사태에 금융위기 수준 악화",https://n.news.naver.com/mnews/article/055/000...
3,여행플랫폼 만족도 1위 ‘여기어때’·고객응대 최하점 받은 ‘이곳’ 최하위,https://n.news.naver.com/mnews/article/009/000...
4,"한은 ""환율 전망 어렵지만, 금융기관 건전성에 큰 영향 없어""",https://n.news.naver.com/mnews/article/421/000...


In [78]:
# lis = bs_obj.find_all("div", {"class":"sa_text"})

# title = []
# titleLink= []
# for li in lis:
#     title.append(li.a.text.replace("\n", ""))
#     titleLink.append(li.a["href"])
# len(title), len(titleLink)
# naver_headline_link = pd.DataFrame({"title":title, "titleLink":titleLink})
# naver_headline_link

- 경제 섹션을 기준으로 작성한 코드가 여러 섹션에 적용되어야 함
- 다른 섹션에 대해 test하지 않으므로 반복 진행하다가 문제(error)가 생기면 해당 기사는 pass 후, 수집하지 않음
- 반복적으로 많은 내용 수집할때는 예외처리 해야함 : 예외가 생기는 컨텐츠는 pass 처리

In [ ]:
# divs에서 전체 title과 link 리스트에 저장 :예외처리 포함

In [81]:
title=[]
titleLink=[]
for li in lis:
    try:
        title.append(li.a.text.replace("\n", ""))
        titleLink.append(li.a["href"])
    except:
        print("예외 발생")
len(title), len(titleLink)

(46, 46)

In [ ]:
# 헤드라인 뉴스에서 각 topic 추출

#### 수집한 데이터를 데이터프레임으로 생성하고, 파일로 저장

In [86]:
tmp_section_data=pd.DataFrame({"기사제목":title,"기사링크":titleLink, "section":"경제"})
tmp_section_data.to_csv("./crawl_data/tmp_section_data.csv", index=0, encoding="utf-8-sig")

############################################################################

#### 지금까지 한 내용을 결합해서 함수로 구성
- 현재 경제 섹션 1개에 대해서 처리했는데
- 다른 섹션에도 적용하기 위해 함수로 구성
- 함수명 : get_topic(url)
      - 섹션별 수집한 데이터를 딕셔너리로 구성 후 반환
- 반환된 딕셔너리를 데이터프레임에 추가해서 최종 데이터프레임 완성
      
      - 빈 DF를 생성해놓고 여기에 항목별로 concat하는 방식
      - 
  

#### 참고
- 접속할 때 자동화 봇으로 간주하고 연결을 끊는 경우가 발생  
- 서버에 요청 시 header 구성해서  
- 내 웹 브라우저의 User-Agent 정보 입력해서  
- bot이 아님을 증명해야 함  
- header 확인  
    - 개발자 도구 (F12)  
    - Network 탭에서, 아래 왼쪽에 있는 이름에서 하나 선택하면 '헤더' 탭 보임
    - '헤더' 탭 / '요청 헤더' 맨 아래에 User-Agent 확인  
- {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'}

In [5]:
import requests

In [6]:
# 함수로 구성
# 섹션과 url 전달 받아서, 헤드라인 topic의 제목, 링크, 섹션을 추출하여
# 딕셔너리로 만들어 반환하는 함수
def get_topic(url, section):
    # {1} 요청 후 응답 받고 파싱객체 생성
    # 요청 시 전달할 header : 딕셔너리루 구성(key:value, 모두 문자열)
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"}
    res = requests.get(url, headers=headers) # 응답객체를 반환 :해당 객체가 복잡하므로 bs4가 인식할 수 없음
    html = res.text

    # 파싱 객체
    bs_obj=BeautifulSoup(html, "html.parser")
    
    # 파싱
    lis = bs_obj.select("ul.sa_list div.sa_text")

    # 컨텐츠 추출
    title=[]
    titleLink=[]
    for li in lis:
        try:
            title.append(li.a.text.replace("\n", ""))
            titleLink.append(li.a["href"])
        except:
            print("예외 발생")
    return ({"기사제목":title,"기사링크":titleLink, "분류":section})

In [7]:
section = section_menu_df["menu"][2]
link = section_menu_df["link"][2]
section, link
test_df = pd.DataFrame(get_topic(link, section))
test_df.head()

('경제', 'https://news.naver.com/section/101')

,기사제목,기사링크,분류
0,"우리금융, 윤리경영실 신설...검찰 출신 이동수 변호사 영입(종합)",https://n.news.naver.com/mnews/article/119/000...,경제
1,"고려아연, 최윤범 회장 가족회사 동원 집중투표 청구 논란",https://n.news.naver.com/mnews/article/001/001...,경제
2,산업연 “트럼프 보편관세 부과시 한국의 대미 수출 최대 13.1% 감소할 것”,https://n.news.naver.com/mnews/article/016/000...,경제
3,"통영시, 국내 1호 관광형 기회발전특구 지정…도산면 복합관광개발",https://n.news.naver.com/mnews/article/421/000...,경제
4,'강달러 지속' 환율 5거래일 연속 1450원↑…1465원도 돌파(종합),https://n.news.naver.com/mnews/article/421/000...,경제


In [8]:
# 함수 테스트 : 사회 섹션도 url과 섹션 전달하고 반환 결과 확인
pd.DataFrame(get_topic(section_menu_df["link"][3], section_menu_df["menu"][3])).head()

,기사제목,기사링크,분류
0,도입 코앞인데 '빨간불' 켜진 AI 디지털교과서…지위 격하되나,https://n.news.naver.com/mnews/article/008/000...,사회
1,"공조본 “尹대통령 측, 오후 6시 현재 연락 없어”...조사 무산",https://n.news.naver.com/mnews/article/023/000...,사회
2,"김형두 헌법재판관 ""국회 측의 계엄사령관 포고령 등으로 증거 갈음""",https://n.news.naver.com/mnews/article/586/000...,사회
3,"""부모 욕해서"" 20대가 던진 폰에, 생후 10개월 아기 두개골 함몰",https://n.news.naver.com/mnews/article/025/000...,사회
4,"경희대, 전체 모집인원 45% 정시 선발…자유전공학부 확대[2025 정시특집]",https://n.news.naver.com/mnews/article/277/000...,사회


#### 데이터프레임으로 저장

- 6개 섹션의 모든 topic을 하나의 데이터프레임에 저장
    - df와 df를 병합하는 방법 사용: {1} 빈데이터 프레임 생성 후, {2} get_topic함수 호출해서 dict 반환, {3} df로 구성
        - 빈 데이터프레임에 반환 data df를 concat 진행

In [11]:
# 페이지 구조(포맷)이 동일하게 보이는 6개의 섹션에 대해서만 수집 진행
# 6번 반복하면서 각 섹션의 결과 dict를 반환 후, df로 구성하는 코드
all_topic_df = pd.DataFrame({
                "기사제목":[],
                "기사링크":[],
                "분류":[],
                })

for i in range(1, 7):
    tmpDF=pd.DataFrame(get_topic(section_menu_df["link"][i], section_menu_df["menu"][i]))
    all_topic_df = pd.concat([all_topic_df, tmpDF], axis=0, ignore_index=True)
all_topic_df.to_csv("./crawl_data/naver_news_all_topic.csv", index=0, encoding="utf-8-sig")

### (2) 각 기사 상세 내용 추출 : sub 페이지 내용 추출
- 언론사, 기사제목, 작성 날짜 시간, 기사 내용, 작성자

In [14]:
all_topic_df=pd.read_csv("./crawl_data/naver_news_all_topic.csv")
all_topic_df

,기사제목,기사링크,분류
0,"박찬대 “한덕수, 국회 헌법재판관 임명동의안 처리 즉시 임명해야”",https://n.news.naver.com/mnews/article/056/001...,정치
1,"국정협의체, '한덕수 탄핵' 변수…첫 회의부터 불투명",https://n.news.naver.com/mnews/article/277/000...,정치
2,"국민의힘, 헌법재판관 임명안 표결 불참 결정",https://n.news.naver.com/mnews/article/018/000...,정치
3,"민주, 명태균 녹취 공개…""윤에 대우조선 파업 당장 진압하라고 보고""",https://n.news.naver.com/mnews/article/003/001...,정치
4,"민주당 ""윤 대통령, 수사거부로 시간끌기""…한덕수 탄핵은 유보",https://n.news.naver.com/mnews/article/374/000...,정치
...,...,...,...
264,"트럼프 취임 목전인데…""한국 대응은 마비 상태""",https://n.news.naver.com/mnews/article/003/001...,세계
265,"지중해서 폭발후 침몰 러시아 화물선주 ""테러행위"" 주장",https://n.news.naver.com/mnews/article/001/001...,세계
266,"포드부터 암호화폐 기업까지, 트럼프 취임식 후원 나선 기업들",https://n.news.naver.com/mnews/article/366/000...,세계
267,의회 폭동 이유 트럼프 지지 철회했던 기업들 속속 취임식 기부,https://n.news.naver.com/mnews/article/003/001...,세계


In [15]:
url=all_topic_df["기사링크"][3]
url

'https://n.news.naver.com/mnews/article/003/0012981678'

In [16]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"}
res = requests.get(url, headers=headers)
html = res.text
bs_obj = BeautifulSoup(html, 'html.parser')
# bs_obj

In [17]:
# 언론사 : a태그의 클래스를 이용해서 추출 예정, 이미의 속성값 title을 활용해서 언론사 이름 추출 예정
# <a href="http://www.edaily.co.kr/" class="media_end_head_top_logo">
# 			<img width="" height="32" alt="이데일리" title="이데일리" class="media_end_head_top_logo_img light_type _LAZY_LOADING _LAZY_LOADING_INIT_HIDE _LAZY_LOADING_ERROR_HIDE" style="" src="https://mimgnews.pstatic.net/image/upload/office_logo/018/2020/09/15/logo_018_6_20200915185838.png">
# 			<span class="media_end_head_top_logo_text light_type _LAZY_LOADING_ERROR_SHOW" style="display: none;">이데일리</span>
# 			<img width="" height="32" alt="이데일리" title="이데일리" class="media_end_head_top_logo_img dark_type _LAZY_LOADING _LAZY_LOADING_INIT_HIDE _LAZY_LOADING_ERROR_HIDE" style="display: inline;" src="https://mimgnews.pstatic.net/image/upload/office_logo/018/2020/09/15/dark_logo_018_6_20200915145317.png">
# 			<span class="media_end_head_top_logo_text dark_type _LAZY_LOADING_ERROR_SHOW" style="display: none;">이데일리</span>
# 		</a>
paper = bs_obj.find("a", {"class":"media_end_head_top_logo"}).select_one("img")["title"]
paper

'뉴시스'

In [18]:
# 기사 제목
# <div class="media_end_head_title">
# 		<h2 id="title_area" class="media_end_head_headline"><span>트럼프 2.0시대 투자법은? SC제일은행 “미국 주식·금 늘리고 현금 축소”</span></h2>
# 	</div>
title = bs_obj.select_one("div.media_end_head_title span").text
title = bs_obj.select_one("h2#title_area span").text

title

'민주, 명태균 녹취 공개…"윤에 대우조선 파업 당장 진압하라고 보고"'

In [19]:
# 작성 일자 및 시간
# <span class="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME" data-date-time="2024-12-26 10:34:17" data-date-time-age-in-minutes="34">2024.12.26. 오전 10:34</span>
bs_obj.find("span", {"class":"media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}).text
# 아래 css 선택자는 .media_end_head_info_datestamp_time인 태그를 찾고, 자손 객체 _ARTICLE_DATE_TIME를 찾으라는 의미가 되어버림
print("""bs_obj.select_one(".media_end_head_info_datestamp_time _ARTICLE_DATE_TIME").text""")

# 클래스값에 공백이 있는 경우
# {1} 공백이전까지만 매칭
bs_obj.select_one(".media_end_head_info_datestamp_time").text
# {2} 공백자리에 .을 추가(정확도가 1번보다 높음)
datetime = bs_obj.select_one(".media_end_head_info_datestamp_time._ARTICLE_DATE_TIME").text


'2024.12.26. 오전 11:13'

bs_obj.select_one(".media_end_head_info_datestamp_time _ARTICLE_DATE_TIME").text


'2024.12.26. 오전 11:13'

In [20]:
# 기사 내용
# article 태그 id: dic_area
writing=bs_obj.select_one("#dic_area").text.replace("\n", "")
writing

'"이영호 부사장 보고서 전달 받아 윤·김에 보고""내가 보고한 직후 한덕수 총리가 긴급회의 소집"[창원=뉴시스] 차용현 기자 = 김건희 여사 공천 개입 의혹 핵심 당사자인 명태균 씨가 14일 오후 영장실질심사를 받기 위해 경남 창원시 성산구 소재 창원지방법원에 출석하고 있다. 2024.11.14. con@newsis.com[서울=뉴시스]조재완 기자 = 윤석열 대통령 부부 공천개입 의혹 핵심 당사자인 명태균 씨가 2022년 대우조선 하청업체 파업에 대한 정부 대응에 적극 개입했다고 과시하는 녹취가 공개됐다. 민주당은 26일 명 씨가 그해 7월 파업 중이었던 경남 거제 조선소에 가는 도중 지인과 이 같은 내용으로 대화했다며 명 씨 육성이 담긴 녹취 파일을 공개했다.녹취에서 명 씨는 대우조선 측이 제출한 보고서를 토대로 파업을 강경 진압해야 한다는 취지로 윤 대통령과 김건희 여사에게 보고했다고 발언했다. 자신이 보고한 직후 파업 대응을 위해 한덕수 국무총리가 긴급회의를 소집했다고도 말했다. 명 씨는 "조선소고 뭐고 내용을 잘 모른다. 거기 문제가 심각한데 지난주 대통령한테 내가 보고를 했다"며 "이영호 부사장인가 내가 보고서를 만들어달라고 했다. (이 부사장이) 만들어주더라고"라고 말했다.이어 "내가 보고하고 나서 한 총리가 긴급 소집한 것 아니냐. 또 내가 보고했지. 당장 진압하라고"라고 말했다. 그는 "대통령이 보고해달라고 해서 보고를 했고, 보고하니까 그날 바로 긴급소집하더라"며 "아래(그저께)는 한동훈 법무부 장관하고 다 불러가지고"라고 했다. 또 "데모하는 놈은 150명이고 거기 하청 일하는 놈은 만 명인데 150명 때문에 만 명이 다 죽게 생겼던데"라며 "(피해 규모가) 그게 지금 5700억원 해가지고 이래저래 하면 7000억원이 된다는데 말이 7000억원이지"라고 했다. 명 씨는 "하여튼 내가 대통령하고 사모님한테 이야기한 게 있어서 보고를 올렸으니까 내가 가서 눈으로 쳐다보기라도 해야지"라며 "갔다와야 나중에 할 말이라도 있지"라고 했다. 민주당은 

In [21]:
# 작성자
writer = bs_obj.select_one(".byline_s").text
writer

'조재완 기자(wander@newsis.com)'

In [ ]:
# 좋아요 수: 동적 태그임(html 소스로 전달되지 않고 js코드로 전달됨 : 태그 내 content는 실시간 변화)
# bs4로 파싱 불가
# <span class="u_likeit_text _count">추천</span>
# bs_obj.select_one("u_likeit_text _count num").text
# bs_obj.find("span", {"class":"u_likeit_text _count num"}).text

#### sub 페이지 내용 추출하는 함수 작성

In [47]:
# sub 페이지 추출 함수
def get_sub_news_info(url):
    try:
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"}
        res = requests.get(url, headers=headers)
        html=res.text
        bs_obj=BeautifulSoup(html, "html.parser")

        # 데이터 자장용 빈리스트 생성
        paper_list, title_list, datetime_list, writing_list, writer_list = [[]for _ in range(5)]
        # 데이터 추출
        # 언론사
        paper_list.append(bs_obj.find("a", {"class":"media_end_head_top_logo"}).select_one("img")["title"])
        # 기사제목
        title_list.append(bs_obj.select_one("h2#title_area span").text)
        # 기사입력 날짜 및 시간
        datetime_list.append(bs_obj.select_one(".media_end_head_info_datestamp_time._ARTICLE_DATE_TIME").text)
        # 기사 내용
        writing_list.append(bs_obj.select_one("#dic_area").text.replace("\n", ""))
        # 작성 기자
        if (bs_obj.select_one(".byline_s") == None):
            writer_list.append("없음")
        else:
            writer_list.append(bs_obj.select_one(".byline_s").text)
    except Exception as e:
        print("오류 발생 :", e)
    
    # 지정된 list 내 원소수 일치여부 확인
    print(len(paper_list), len(title_list), len(datetime_list), len(writing_list), len(writer_list))
    sub_new_dict = {"paper":paper_list, 
                                 "title":title_list, 
                                 "datetime":datetime_list, 
                                 "writing":writing_list, 
                                 "writer":writer_list}
    return sub_new_dict

In [33]:
# 함수 테스트 : 1개의 url 전달해서 반환 결과 확인f
for url in all_topic_df["기사링크"][:5]:
    get_sub_news_info(url)

1 1 1 1 1


{'paper': ['KBS'],
 'title': ['박찬대 “한덕수, 국회 헌법재판관 임명동의안 처리 즉시 임명해야”'],
 'datetime': ['2024.12.26. 오전 9:48'],
 'writing': ["더불어민주당 박찬대 원내대표는 한덕수 대통령 권한대행 겸 국무총리를 향해 국회 본회의에서 헌법재판관 3명에 대한 임명동의안이 처리되면 곧바로 임명할 것을 촉구했습니다.  박 원내대표는 오늘(26일) 국회에서 열린 정책조정회의에서 “한덕수 총리는 국회가 헌법재판관 임명동의안을 이송하는 즉시 일분일초도 지체하지 말고 바로 임명하라”고 말했습니다.  이어 “헌법재판관을 임명하지 않는 건 명백한 위헌 행위이고 12·3 내란 사태를 조속히 끝내지 않겠다는 심각한 반국가 행위”라며 “역사의 반역자, 을사오적의 길을 걷지 말라”고도 말했습니다.  박 원내대표는 “5천만 국민이 한 총리를 지켜보고 있다”며 “오늘까지가 인내할 수 있는 마지막 시한”이라고 덧붙였습니다.  앞서 민주당은 지난 24일 한 대행에 대한 탄핵소추안을 제출하려다 막판에 결정을 뒤집고 26일까지 기다려보겠다며 헌법재판관 임명을 촉구한 바 있습니다.  한편, 국민의힘 새로운 비대위 출범과 관련해 박 원내대표는 “검찰총장 출신 내란수괴로 탄핵된 윤석열과 검사 출신 원내대표 권성동, 검사 출신 비대위원장 권영세는 검사동일체를 넘어선 내란 동일체‘라고 비판했습니다.  그러면서 ”국민의힘이 헌법과 법률에 반해 헌법재판관 임명을 반대하는 이유는 이들이 내란 수괴와 한 몸이기 때문“이라며 ”헌정질서를 파괴하는 공당의 자격이 없다“고 주장했습니다.[사진 출처 : 연합뉴스]■ 제보하기▷ 전화 : 02-781-1234, 4444▷ 이메일 : kbs1234@kbs.co.kr▷ 카카오톡 : 'KBS제보' 검색, 채널 추가▷ 네이버, 유튜브에서 KBS뉴스를 구독해주세요!"],
 'writer': ['손서영 (bellesy@kbs.co.kr)']}

1 1 1 1 1


{'paper': ['아시아경제'],
 'title': ["국정협의체, '한덕수 탄핵' 변수…첫 회의부터 불투명"],
 'datetime': ['2024.12.25. 오후 5:51'],
 'writing': ['野, 한덕수 탄핵소추안 발의 압박내일 첫 회의인데 협의 진전 없어"향후 일정도 차례로 순연 가능성"비상계엄 사태에 따른 탄핵 정국의 혼란을 수습하기 위해 여야와 정부가 머리를 맞대기로 했지만 국정안전협의체는 첫발을 떼기 전부터 차질을 빚게 됐다. 정부를 대표하는 한덕수 대통령 권한대행 겸 국무총리에 대해 야당이 탄핵소추를 예고하고 나선 만큼 당장 26일로 예정된 첫 회의부터 개최 여부가 불투명하다.한덕수 대통령 권한대행 국무총리가 연말연시를 맞아 24일 서울 중부소방서를 방문해 현황 보고를 받고 있다. 연합뉴스국회 관계자는 "협의체 가동을 위한 실무진 회의가 늦어도 오늘까진 이뤄졌어야 하는데 그러지 못했다"며 "내일로 예정됐던 협의체 출범 등 향후 일정도 차례로 순연될 가능성이 크다"고 25일 밝혔다. 국민의힘 관계자도 "실무 단계에서 논의가 전혀 진행되지 않고 있다"며 "현재로선 협의체 개최가 매우 불투명하다"고 했다.야권에선 국민의힘이 권영세 비상대책위원장을 지명하고 이에 따른 지도부 개편을 진행 중인 게 영향을 줬다는 입장이다. 박성준 민주당 원내수석부대표는 MBC 라디오에서 "(의제 등) 실무 협의를 하자고 했는데 국민의힘이 준비가 안 됐다"며 "내일 국정안정 협의체 첫발을 떼기가 상당히 어려울 것"이라고 내다봤다.25일 서울 여의도 국회의사당 정문에 "총리마저 탄핵합니까? 안정이 우선입니다"라고 적힌 국민의힘의 현수막이 걸려 있다. 연합뉴스정치권에선 한덕수 대행에 대한 탄핵 문제가 협의체 출범에 차질을 초래했다고 보는 시각이 많다. 민주당은 26일로 예정된 본회의에서 국회 추천 몫 헌법재판관 후보자 3명에 대한 임명동의안을 통과시킨 뒤 한 대행이 곧바로 임명 절차를 밟지 않을 경우 그에 대한 탄핵소추안을 발의하겠다고 엄포를 놓은 상황이다.실제 

1 1 1 1 1


{'paper': ['이데일리'],
 'title': ['국민의힘, 헌법재판관 임명안 표결 불참 결정'],
 'datetime': ['2024.12.26. 오후 12:04'],
 'writing': ['26일 오전 의원총회서 결정…의원 전원 불참에 만장일치권성동 “검사가 판사 고르는 꼴…탄핵심판 무효화될 수도”[이데일리 김응열 기자] 국민의힘이 헌법재판관 임명동의안 표결에 불참한다.권성동 국민의힘 대표 권한대행 겸 원내대표가 26일 오전 서울 여의도 국회에서 열린 의원총회에서 모두발언을 하고 있다. (사진=노진환 기자)권성동 국민의힘 대표 권한대행 겸 원내대표는 26일 오전 서울 국회에서 진행한 의원총회를 마친 뒤 취재진과 만나 “한덕수 대통령 권한대행에게 헌법재판관 임명 권한이 없다는 입장을 여전히 견지하고 있다”며 “그런 차원에서 헌법재판관 인사청문회에 참여하지 않았고 표결도 참여하지 않기로 결정했다”고 밝혔다.권 권한대행은 “당론 결정 절차는 밟지 않았으나 불참하자는 의사에 의원들이 만장일치로 동의했다”고 설명했다.야권 주도로 헌법재판관 임명동의안이 본회의를 통과할 경우 권한쟁의 심판이나 효력정지 가처분 등을 신청할지 여부에 관해서는 “한 권한대행에게 헌법재판관 임명권이 없다고 생각해서, 당에선 관련한 후속 절차를 밟을 계획이 없다”고 언급했다.권 권한대행 의원총회 모두발언에서도 국회 몫 헌법재판관 3명의 임명을 한 권한대행이 할 수 없다는 점을 거듭 강조했다. 권 권한대행은 “국회가 대통령 탄핵 심판의 소추인인데 국회가 헌법재판관을 추천하는 건 검사가 판사를 고르는 것”이라고 지적했다.또 “과거 윤석열 대통령이 검찰총장이던 시절 당시 추미애 법무부장관이 윤 총장 징계처분을 하고나서 징계위원 한 명을 추가 임명했는데 서울고등법원이 헌법상 적정 절차 위반이라고 징계 청구 자체를 무효라고 판시한 바 있다”며 “국회가 추천한 헌법재판관 3인의 임명을 강행하면 탄핵심판 자체가 무효화될 수 있다”고 주장했다.더불어민주당이 추진하는 한 권한대행 탄핵을 두고는 “대통

1 1 1 1 1


{'paper': ['뉴시스'],
 'title': ['민주, 명태균 녹취 공개…"윤에 대우조선 파업 당장 진압하라고 보고"'],
 'datetime': ['2024.12.26. 오전 11:13'],
 'writing': ['"이영호 부사장 보고서 전달 받아 윤·김에 보고""내가 보고한 직후 한덕수 총리가 긴급회의 소집"[창원=뉴시스] 차용현 기자 = 김건희 여사 공천 개입 의혹 핵심 당사자인 명태균 씨가 14일 오후 영장실질심사를 받기 위해 경남 창원시 성산구 소재 창원지방법원에 출석하고 있다. 2024.11.14. con@newsis.com[서울=뉴시스]조재완 기자 = 윤석열 대통령 부부 공천개입 의혹 핵심 당사자인 명태균 씨가 2022년 대우조선 하청업체 파업에 대한 정부 대응에 적극 개입했다고 과시하는 녹취가 공개됐다. 민주당은 26일 명 씨가 그해 7월 파업 중이었던 경남 거제 조선소에 가는 도중 지인과 이 같은 내용으로 대화했다며 명 씨 육성이 담긴 녹취 파일을 공개했다.녹취에서 명 씨는 대우조선 측이 제출한 보고서를 토대로 파업을 강경 진압해야 한다는 취지로 윤 대통령과 김건희 여사에게 보고했다고 발언했다. 자신이 보고한 직후 파업 대응을 위해 한덕수 국무총리가 긴급회의를 소집했다고도 말했다. 명 씨는 "조선소고 뭐고 내용을 잘 모른다. 거기 문제가 심각한데 지난주 대통령한테 내가 보고를 했다"며 "이영호 부사장인가 내가 보고서를 만들어달라고 했다. (이 부사장이) 만들어주더라고"라고 말했다.이어 "내가 보고하고 나서 한 총리가 긴급 소집한 것 아니냐. 또 내가 보고했지. 당장 진압하라고"라고 말했다. 그는 "대통령이 보고해달라고 해서 보고를 했고, 보고하니까 그날 바로 긴급소집하더라"며 "아래(그저께)는 한동훈 법무부 장관하고 다 불러가지고"라고 했다. 또 "데모하는 놈은 150명이고 거기 하청 일하는 놈은 만 명인데 150명 때문에 만 명이 다 죽게 생겼던데"라며 "(피해 규모가) 그게 지금 5700억원 해가지고 이래저래 하면 7000억원이 된

1 1 1 1 1


{'paper': ['SBS Biz'],
 'title': ['민주당 "윤 대통령, 수사거부로 시간끌기"…한덕수 탄핵은 유보'],
 'datetime': ['2024.12.25. 오후 5:21'],
 'writing': ['[야6당 의원들이 12일 오전 서울 여의도 국회 의안과에 윤석열 대통령 탄핵 소추안을 제출하고 있다. (사진=연합뉴스)]더불어민주당 등 야권은 오늘(25일) 윤석열 대통령이 고위공직자범죄수사처의 2차 출석 요구에 불응한 것과 관련해 구속수사를 촉구했습니다.한덕수 대통령 권한대행 국무총리에 대해선 국회 몫 헌법재판관 3인을 임명할 때까지 탄핵을 유보하기로 했습니다.한민수 민주당 대변인은 이날 브리핑에서 "내란수괴 윤석열이 또 다시 공수처의 출석 요구를 거부했다"고 지적했습니다.이어 공수처에 "반복되는 출석요구와 수사 지연은 국민의 불신만 초래할 뿐"이라며 "사건의 엄중함을 직시하고 즉시 내란수괴에 대한 체포와 구속수사를 진행하라"고 촉구했습니다.민주당은 전날 한 권한대행 국무총리 탄핵안을 발의 직전에 유보했습니다.대통령 탄핵절차를 위해선 앞서\xa0한 권한대행이 사실상 거부한 특검법 공포보다 헌법재판관 임명이 더 중요하다고 판단한 겁니다.민주당은 오는 27일 오전까지 상황을 지켜보고, 이때까지도 한 권한대행이 헌법재판관을 임명하지 않는다면 같은 날 한 권한대행에 대한 탄핵안을 발의할 것으로 예상됩니다.탄핵안은 보고된 시점부터 24시간 이후 72시간 이내에 표결돼야 하므로\xa0이 경우 다음 주 월요일인 30일에 탄핵안 표결이 이뤄질 것으로 보입니다.한 권한대행 탄핵안이 본회의를 통과할 경우, 최상목 부총리 겸 기획재정부 장관이 그 뒤를 이어 대통령 권한대행을 맡게 됩니다.이와 관련해 박범계 민주당 의원은 한 라디오 인터뷰에서 "비상계엄 전 (국무위원이 모였을 때) 계엄에 가장 강하게 반대한 점 등을 보면 한 권한대행보다는 낫지 않겠나"라고 말하기도 했습니다.당신의 제보가 뉴스로 만들어집니다.SBS Biz는 여러분의 제보를 기다리고 있습니다.홈

#### 함수에서 반환된 값으로 데이터프레임에 추가해서 최종 데이터프레임 생성

In [48]:
all_sub_news_info_df = pd.DataFrame({"paper":[], 
                                 "title":[], 
                                 "datetime":[], 
                                 "writing":[], 
                                 "writer":[]
                                 })
all_sub_news_info_df
for url in all_topic_df["기사링크"]:
    tmp_df = pd.DataFrame(get_sub_news_info(url))
    all_sub_news_info_df=pd.concat([all_sub_news_info_df, tmp_df], axis = 0, ignore_index=True)
# 여기서 나오는 에러값은 요소의 길이중 1이 아닌 것이 있다는 얘기, writer가 공란인 경우가 있음
all_sub_news_info_df

,paper,title,datetime,writing,writer


1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1
1 1 1 1 1


,paper,title,datetime,writing,writer
0,KBS,"박찬대 “한덕수, 국회 헌법재판관 임명동의안 처리 즉시 임명해야”",2024.12.26. 오전 9:48,더불어민주당 박찬대 원내대표는 한덕수 대통령 권한대행 겸 국무총리를 향해 국회 본회...,손서영 (bellesy@kbs.co.kr)
1,아시아경제,"국정협의체, '한덕수 탄핵' 변수…첫 회의부터 불투명",2024.12.25. 오후 5:51,"野, 한덕수 탄핵소추안 발의 압박내일 첫 회의인데 협의 진전 없어""향후 일정도 차례...",장희준 기자 junh@asiae.co.kr
2,이데일리,"국민의힘, 헌법재판관 임명안 표결 불참 결정",2024.12.26. 오후 12:04,26일 오전 의원총회서 결정…의원 전원 불참에 만장일치권성동 “검사가 판사 고르는 ...,김응열(keynews@edaily.co.kr)
3,뉴시스,"민주, 명태균 녹취 공개…""윤에 대우조선 파업 당장 진압하라고 보고""",2024.12.26. 오전 11:13,"""이영호 부사장 보고서 전달 받아 윤·김에 보고""""내가 보고한 직후 한덕수 총리가 ...",조재완 기자(wander@newsis.com)
4,SBS Biz,"민주당 ""윤 대통령, 수사거부로 시간끌기""…한덕수 탄핵은 유보",2024.12.25. 오후 5:21,[야6당 의원들이 12일 오전 서울 여의도 국회 의안과에 윤석열 대통령 탄핵 소추안...,정광윤 기자(jky@sbs.co.kr)
...,...,...,...,...,...
264,뉴시스,"트럼프 취임 목전인데…""한국 대응은 마비 상태""",2024.12.26. 오전 10:28,"尹계엄 이후 정국 혼란에…""가장 필요할 때 대응 못 해""[피닉스=AP/뉴시스] 도널...",김난영 기자(imzero@newsis.com)
265,연합뉴스,"지중해서 폭발후 침몰 러시아 화물선주 ""테러행위"" 주장",2024.12.26. 오전 10:27,러시아 화물선 우르사 메이저[로이터 연합뉴스 자료사진 재판매 및 DB 금지](서울=...,김계환(kp@yna.co.kr)
266,조선비즈,"포드부터 암호화폐 기업까지, 트럼프 취임식 후원 나선 기업들",2024.12.26. 오전 10:26,2021년 1·6 의회 폭동 이후 정치 기부 중단 선언했던 기업들 트럼프 ...,정미하 기자 viva@chosunbiz.com
267,뉴시스,의회 폭동 이유 트럼프 지지 철회했던 기업들 속속 취임식 기부,2024.12.26. 오전 10:18,"""식탁에 앉지 못하면 메뉴판에 오른다"" 속담 따라 줄대기 성황100만 달러 기부하면...",강영진 기자(yjkang1@newsis.com)


In [49]:
all_sub_news_info_df.to_csv("./crawl_data/naver_sub_news_section.csv", index=0, encoding="utf-8-sig")